Test API key

In [4]:
import requests
from requests.auth import HTTPBasicAuth

api_key = 'PLAK4bc733159fb946edb914e69f1224dee0' # Replace with your actual API key
auth = HTTPBasicAuth(api_key, "")

test_url = "https://api.planet.com/data/v1/item-types"
response = requests.get(test_url, auth=auth)

print(response.status_code)
print(response.text)



401
{"message": "Please enter a valid API key.", "errors": []}


In [3]:
import requests
import json
import time
from requests.auth import HTTPBasicAuth
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

# 1. Set Up Authentication
api_key = 'PLAK4bc733159fb946edb914e69f1224dee0'  # Replace with your actual API key
auth = HTTPBasicAuth(api_key, '')

# 2. Define the Area of Interest (AOI)
aoi = {
    "type": "Polygon",
    "coordinates": [
        [
            [-122.522, 37.7045],
            [-122.522, 37.8295],
            [-122.355, 37.8295],
            [-122.355, 37.7045],
            [-122.522, 37.7045]
        ]
    ]
}

# 3. Construct Search Filters
search_filter = {
    "type": "AndFilter",
    "config": [
        {
            "type": "GeometryFilter",
            "field_name": "geometry",
            "config": aoi
        },
        {
            "type": "DateRangeFilter",
            "field_name": "acquired",
            "config": {
                "gte": "2025-01-01T00:00:00Z",
                "lte": "2025-02-01T00:00:00Z"
            }
        },
        {
            "type": "RangeFilter",
            "field_name": "cloud_cover",
            "config": {
                "lte": 0.1
            }
        }
    ]
}

# 4. Perform the Search
search_endpoint = 'https://api.planet.com/data/v1/quick-search'
search_request = {
    "item_types": ["PSScene4Band"],
    "filter": search_filter
}

response = requests.post(search_endpoint, auth=auth, json=search_request)

if response.status_code != 200:
    print("Error during search:", response.status_code, response.text)
    exit()

geojson = response.json()

# Check if search response contains 'features'
if "features" not in geojson:
    print("Unexpected response format:", json.dumps(geojson, indent=4))
    exit()

if not geojson["features"]:
    print("No imagery found for the specified criteria.")
    exit()

# 5. Get Item IDs
item_ids = [item["id"] for item in geojson["features"][:3]]  # Limit to first 3 items
print("Found items:", item_ids)

# 6. Place an Order via Orders API
order_endpoint = "https://api.planet.com/compute/ops/orders/v2"
order_request = {
    "name": "satellite_image_order",
    "source_type": "scenes",
    "products": [
        {
            "item_ids": item_ids,
            "item_type": "PSScene",
            "product_bundle": "analytic_udm2"
        }
    ]
}

order_response = requests.post(order_endpoint, auth=auth, json=order_request)
if order_response.status_code != 200:
    print("Error placing order:", order_response.status_code, order_response.text)
    exit()

order_data = order_response.json()
order_id = order_data["id"]
print(f"Order placed successfully. Order ID: {order_id}")

# 7. Wait for Order to Complete
status_endpoint = f"https://api.planet.com/compute/ops/orders/v2/{order_id}"
while True:
    status_response = requests.get(status_endpoint, auth=auth)
    if status_response.status_code != 200:
        print("Error checking order status:", status_response.status_code, status_response.text)
        exit()

    status_data = status_response.json()
    state = status_data["state"]
    print(f"Order Status: {state}")

    if state in ["success", "failed", "partial"]:
        break

    time.sleep(15)  # Wait 15 seconds before checking again

# 8. Download the Imagery
if state == "success":
    print("Order completed successfully! Downloading images...")
    for result in status_data.get("_links", {}).get("results", []):
        download_url = result["location"]
        file_name = result["name"].split("/")[-1]

        image_response = requests.get(download_url, auth=auth)
        with open(file_name, "wb") as file:
            file.write(image_response.content)

        print(f"Downloaded: {file_name}")

        # Display the image if it's a TIFF
        if file_name.endswith(".tif"):
            image = Image.open(BytesIO(image_response.content))
            plt.imshow(image)
            plt.axis('off')  # Hide axes
            plt.show()

elif state == "partial":
    print("Order partially completed. Some images may be missing.")
else:
    print("Order failed.")

Error during search: 401 {"message": "Please enter a valid API key.", "errors": []}
Unexpected response format: {
    "message": "Please enter a valid API key.",
    "errors": []
}


KeyError: 'features'